In [102]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator #-> for major + minor ticks (Cf: https://matplotlib.org/stable/gallery/ticks/major_minor_demo.html#major-and-minor-ticks)
import pandas as pd
from copy import deepcopy
import time

In [103]:
from laser_dataset import FileReader, LaserDataInteractor

In [219]:
"""
!!! This is a modified version for iHub 2023 data !!!

It does following:
    * Load the desired dataset
    * Compute the groudtruth segmentation (based on the Versuchstabelle and proper synchronization)
    * Plot all signals (ch1, ch2, ch3 and groundtruth)
    * Save the plot 
"""

def save_plots_single_trial(_interactor, trialNo, path_qass, path_plots, path_delay, fontsize=16):
    """
    
    The plot is saved as png, i.e. axis labels and titles should be scaled/modified in this function 
    (these settings cannot be changed in Latex once the plot is generated).
    
    Parameters
    ----------
        _interactor: laser data interactor class
        trialNo: str
            Trial number used to save the plots (same as DEWETRON name)
        path_data: str
            Path to the raw data
        path_plots: str
            Path to the directory for saving the plot
    
    Comments
    --------
        * Saving plots as pgfplots coordinates (for Latex) failed due to the large number of data points 
          (thus data size, 1.5GB per channel)
        * Some settings should be adjusted and modified depending on the usage 
          (e.g. line width, font size for axis labels etc)
        * Higher resolution does not yield perceptible improvement in resolution, yet this can be also 
          increased/decreased 
        * Resulting data size (per channel) with the current setup = around 100KB 
    
    """    
    # Params
    fs_qass = 6.25*10**6 # [Hz]
    nfft = 2048
    l_seg = nfft # Length of each STFT segment
    l_overlap = int(l_seg/2)
    fmin = 0 # [Hz]
    fmax = 400*10**3#1000*10**3 # [Hz]
    
    # Load the raw data -> spedify the row
    _interactor.df_row = trialNo
    #--- load the raw data
    # !!! Ch.2 of trial 01_0_1_Spalt_pneu is not available !!!!!
    s1 = _interactor.load(chNo=1, path=path_qass)    
    s2 = _interactor.load(chNo=2, path=path_qass)
    s3 = _interactor.load(chNo=3, path=path_qass)
    S = [s1, s2, s3]
    
    # t_start & t_end and the system delay
    t_start_camera = _interactor.df_row['tstart [s]'].item() #[s]
    t_end_camera = _interactor.df_row['tend [s]'].item() #[s]
    delay_df = pd.read_csv(f'{path_delay}/delays_QASS.csv')
    delay = 26*10**-3 + delay_df['delay[s]'][delay_df['DEWETRON'] == trialNo].item() #[s]
    t_start_qass = t_start_camera - delay
    t_end_qass = t_end_camera - delay
    print(f'delay = {round(delay*10**3, 2)}ms')
    print(f't_start_qass = {round((t_start_qass)*10**3, 2)}ms')
    print(f't_end_qass = {round((t_end_qass)*10**3, 2)}ms')
    
    # Iterate
    for idx in [2]:#range(3):
        
        # Channel number
        chNo = idx + 1
        
        # Instantiate
        fig, ax = plt.subplots(1, 1, sharex = False, sharey = False)
        # Figure size
        fig.set_figwidth(8.8*0.45)#(6.5)
        fig.set_figheight(3.5*0.45)#(4)

        # Spectrogram
        Pxx, freqs, bins, im = ax.specgram(
            S[idx], 
            NFFT=nfft, 
            Fs = fs_qass, 
            noverlap = l_overlap, 
            mode='psd', scale='dB',
            cmap = 'rainbow', 
            sides = 'onesided', 
            scale_by_freq=True, # same as matlab apparently 
            vmax = 10, vmin = -140
        )
        
        #----- Make the plot prettier
        # Limit the time range
        ax.set_xlim(t_start_qass, t_end_qass)#(0.1, 1.2)
        ax.set_xticks(np.arange(t_start_qass, t_end_qass+0.05, 0.2))
        ax.set_xticklabels([])#(np.around(np.arange(0, t_end_qass-t_start_qass+0.05, 0.2), 2))
        # Limit the freq. range
        ax.set_ylim(fmin, fmax)
        # Ticks + labels: y
        # -> (a) for up to 1 MHz
        # ax.set_yticks(np.array([0, 50, 250, 500, 750, 1000])* 10**3)
        # ax.set_yticklabels(['0', '50k', '250k', '500k', '750k', '1M'], fontsize=fontsize)
        # -> (b) for up to 400 kHz
        ax.set_yticks(10**3* np.arange(0, 401, 100))
        ax.set_yticklabels([])#(['0', '100k', '200k', '300k', '400k'], fontsize=fontsize)
        ax.yaxis.set_minor_locator(AutoMinorLocator(2)) #-> creates a single minor tick between major ticks        
        
        # Title
        #ax.set_title(f'Ch.{chNo}', fontsize=fontsize)
        ax.grid(True)

        # Color bar:Cf. https://matplotlib.org/stable/gallery/ticks/colorbar_tick_labelling_demo.html
        cbar = fig.colorbar(im, ax = ax)
        cbar.set_ticks(ticks=[-135, -90, -45, 0], labels=[])
    
        # Margins (add an offset to the figure)
        plt.margins(x = 0.0, y = 0.03)

        # (4) Save
        fname_plots = '{}/{}_ch{}.png'.format(path_plots, trialNo, chNo)
        # Parameters for savefig
        # dpi > 150 does not yield any perceptible difference in resolution 
        # bbox_inches: bounding box to determine the area to be saved => use 'tight', otherwise including a weird white space
        # pad_inches: margine to be padded, if bbox_inches == 'tight'
        plt.savefig(fname_plots, dpi = 300, bbox_inches = 'tight', pad_inches = 0.05) 

        # Close the figure
        plt.close()
        #plt.show()
        
    print('Saved!')
    
    

## (0) Load the meta data

In [220]:
path_meta = '/Volumes/Sandisk_SD/Work/IZFP/Laser/05_2023_iHub/00_Versuchsplan_iHub.xlsx'
metadf = pd.read_excel(
    path_meta,
    sheet_name=1,
    usecols="C:G, I, R,S",
    nrows=71
)
# Keep only the ones / rows for which the QASS data are available
metadf = metadf[metadf['QASS'].notna()]

## Save the spectrograms

In [221]:
path_qass = '/Volumes/Sandisk_SD/Work/IZFP/Laser/05_2023_iHub/02_QASS'
path_png = '/Volumes/Sandisk_SD/Work/IZFP/Laser/05_2023_iHub/05_Evaluation/catalog/figures/plots/400kHz'
path_delay = '/Volumes/Sandisk_SD/Work/IZFP/Laser/05_2023_iHub/04_Delays'

In [222]:
start = time.time()

# Instantiate the interactor
interactor = LaserDataInteractor()
interactor.df = metadf

for idx, trialNo in enumerate(['20_0_3_Spalt_pneu']):#enumerate(metadf['DEWETRON'][5:]):
    start_iter = time.time()
    print(f'iter = {idx}, trialNo = {trialNo}')
    save_plots_single_trial(interactor, trialNo, path_qass, path_png, path_delay)
    
    print(f'execution time = {round(time.time()-start_iter, 3)}s')
    print('#==================#')

end = time.time()
print(f'Complete = {round((end - start)/60, 3)}min')

iter = 0, trialNo = 20_0_3_Spalt_pneu
delay = 27.87ms
t_start_qass = 222.93ms
t_end_qass = 1029.13ms
Saved!
execution time = 4.369s
#==================#
Complete = 0.073min


In [121]:
import sys
sys.exit()

SystemExit: 

/Users/sayakokodera/miniforge3/envs/insigna/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print(df['Versuchs-\nNr.'].sort_values)

## Generate the tex files

In [ ]:
# List of the start of each scenarios
breakpoints = ['0_1', '1_1', '7_1', '21_1']


In [ ]:
'%left'

In [ ]:
"""
Example tex file for each page using the command visualizesingletrial:
    
    % left = 1_2
    \subsection*{Trial 1\textunderscore2}
    \begin{figure}[h!]
    \visualizesingletrial{{figures/plots/1_2_ch1.png}}{{figures/plots/1_2_ch2.png}}{{figures/plots/1_2_ch3.png}}{{figures/plots/1_2_ch4.png}}
    \end{figure}
    %
    % right = 1_3
    \subsection*{Trial 1\textunderscore3}
    \begin{figure}[h!]
        \visualizesingletrial{{figures/plots/1_3_ch1.png}}{{figures/plots/1_3_ch2.png}}{{figures/plots/1_3_ch3.png}}{{figures/plots/1_3_ch4.png}}
    \end{figure}
    \clearpage
"""

# For saving files
path_tex = '/Users/sayakokodera/Work/IZFP/Laser/Tex/campaign1/catalog/figures'


def tex_text_single_trial(trialNo):
    text = (r"""\subsection*{{Trial {}\textunderscore{}}}""".format(trialNo.split('_')[0], trialNo.split('_')[1]) +
            '\n' +
            r"""\begin{figure}[h!]""" + '\n' +
            r"""\visualizesingletrial{{figures/plots/{}_ch1.png}}""".format(trialNo) + 
            r"""{{figures/plots/{}_ch2.png}}""".format(trialNo) + 
            r"""{{figures/plots/{}_ch3.png}}""".format(trialNo) + 
            r"""{{figures/plots/{}_ch4.png}}""".format(trialNo) + 
            '\n' +
            r"""\end{figure}"""
            )
    return text
    


def tex_files_single_page(trial_left, trial_right, path_tex, scenarioNo, pageNo):
    """
    Parameters
    ----------
        trial_left: str
            Trial No. of the left figure
        trial_right: str
            Trial No. of the right figure
    """
    print(f'*** New page!: left = {trial_left}, right = {trial_right}')
    
    text_left = tex_text_single_trial(trial_left)
    text_right = tex_text_single_trial(trial_right)

    # Generate a tex file
    text = (f'% left = {trial_left}' + '\n' +
            text_left + '\n' +
            '%' + '\n' + 
            f'% right = {trial_right}' + '\n' +
            text_right + '\n' +
            r"""\clearpage""")

    # Save
    
    fname = f'{path_tex}/figures/scenario_{scenarioNo}_page_{pageNo}.tex'
    f = open(fname, 'w')
    f.write(text)
    f.close() 

    print(f'Scenario {scenarioNo}, page {pageNo}: data saved! as')
    print(fname)
    print('+==================================+')
    

In [ ]:
path_tex = '/Users/sayakokodera/Work/IZFP/Laser/Tex/campaign1/catalog'

tex_files_single_page(df['Versuchs-\nNr.'][0], df['Versuchs-\nNr.'][1], path_tex, 0, 1)

In [ ]:
path_tex = '/Users/sayakokodera/Work/IZFP/Laser/Tex/campaign1/catalog'

# List of the start of each scenarios
breaksets = ['0', '1', '7', '21']


# Initialize
scenarioNo = 0
pageNo = 1

# Iterate over every 2(!!!) trials
for (trial_left, trial_right) in zip(np.array(df['Versuchs-\nNr.'])[0::2], np.array(df['Versuchs-\nNr.'])[1::2]):
    # scenarioNo
    setNo = trial_left.split('_')[0]
    # Increase the scenarioNo at an breakpoint
    # If scenarionNo == 3 -> no more change
    if scenarioNo < len(breaksets) - 1: 
        # Check if the current left trial is one of the breakpoints
        # If True, change increase the scenarioNo & initialize the pageNo
        if int(setNo) >= int(breaksets[scenarioNo+1]):
            scenarioNo += 1
            pageNo = 1
    
    # Generate the tex file
    tex_files_single_page(trial_left, trial_right, path_tex, scenarioNo, pageNo)
    
    
    # Increase the pageNo
    pageNo += 1